<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/notebook/Assembly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install autorch > log.txt
!pip install optuna > log.txt

In [34]:
import joblib
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import autorch
from autorch.function import sp2wt
import optuna

# LOAD DATA

In [35]:
icg_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/icg_col_names.pkl')
c620_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl')
c660_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl')
t651_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/t651_col_names.pkl')
c670_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl')

In [36]:
icg_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/icg_train.csv',index_col=0)
c620_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_train.csv',index_col=0)
c660_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c660_train.csv',index_col=0)
c670_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c670_train.csv',index_col=0)
t651_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/t651_train.csv',index_col=0)
allidx = list(set(icg_df.index)&
      set(c620_df.index)&
      set(c660_df.index)&
      set(c670_df.index)&
      set(t651_df.index))
len(allidx)

1296

# INPUT

In [37]:
# icg
icg_input = icg_df.loc[allidx,icg_c['x']]
icg_input = icg_input.join(c620_df.loc[allidx,'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'])
icg_input = icg_input.join(c660_df.loc[allidx,'Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'])
icg_input = icg_input.join(c620_df.loc[allidx].filter(regex='Receiver Temp'))

# c620
c620_feed = c620_df.loc[allidx,c620_c['x41']]

# t651
t651_feed = t651_df.loc[allidx,t651_c['x41']]

# OUTPUT

In [38]:
c620_op = c620_df.loc[allidx,c620_c['density']+c620_c['yRefluxRate']+c620_c['yHeatDuty']+c620_c['yControl']]
c620_wt = c620_df.loc[allidx,c620_c['vent_gas_x']+c620_c['distillate_x']+c620_c['sidedraw_x']+c620_c['bottoms_x']]
c660_op = c660_df.loc[allidx,c660_c['density']+c660_c['yRefluxRate']+c660_c['yHeatDuty']+c660_c['yControl']]
c660_wt = c660_df.loc[allidx,c660_c['vent_gas_x']+c660_c['distillate_x']+c660_c['sidedraw_x']+c660_c['bottoms_x']]
c670_op = c670_df.loc[allidx,c670_c['density']+c670_c['yRefluxRate']+c670_c['yHeatDuty']+c670_c['yControl']]
c670_wt = c670_df.loc[allidx,c670_c['distillate_x']+c670_c['bottoms_x']]

# CONFIG

In [39]:
config = {
      # c620
      'c620_G':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_G.pkl',
      'c620_F':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_F.pkl',
      'c620_op_min':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c620_op_min.pkl',
      'c620_op_max':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c620_op_max.pkl',

      # c660
      'c660_G':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c660_G.pkl',
      'c660_F':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c660_F.pkl',
      'c660_op_min':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c660_op_min.pkl',
      'c660_op_max':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c660_op_max.pkl',

      # c670
      'c670_M':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c670.pkl',
      
      # col_names
      'icg_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/icg_col_names.pkl',
      'c620_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl',
      'c660_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl',
      'c670_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl',
      
      # Special column (0.9999 & 0.0001)
      'index_9999_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/index_9999.pkl',
      'index_0001_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/index_0001.pkl',

      # sp
      'c620_wt_always_same_split_factor_dict':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c620_wt_always_same_split_factor_dict.pkl',
      'c660_wt_always_same_split_factor_dict':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c660_wt_always_same_split_factor_dict.pkl',
      'c670_wt_always_same_split_factor_dict':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c670_wt_always_same_split_factor_dict.pkl',
          }

# 輸入與輸出的對應關係

In [40]:
# 對應關係1
a = icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]
b = c620_df['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']
a.join(b)

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
081-008,90.0,90.000000
053-017,80.0,79.999992
092-002,90.0,89.999992
040-011,80.0,80.000015
066-017,80.0,80.000015
...,...,...
035-008,90.0,90.000000
116-011,80.0,80.000000
010-008,90.0,90.000000
030-002,90.0,90.000000


In [41]:
#對應關係2
a = icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]
na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
b = c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1)*10000
b.name = 'c660_wt_NA_in_Benzene_ppmw'
a.join(b)

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,c660_wt_NA_in_Benzene_ppmw
081-008,920.0,920.005904
053-017,920.0,920.106315
092-002,980.0,979.888193
040-011,980.0,980.000417
066-017,920.0,920.001935
...,...,...
035-008,920.0,919.997237
116-011,980.0,979.985678
010-008,920.0,920.000279
030-002,950.0,949.996779


In [42]:
#對應關係3
a = icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]
b = c660_df['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%']*10000
a.join(b)

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%
081-008,5.000003,5.000003
053-017,9.999991,9.999991
092-002,9.999974,9.999974
040-011,9.999931,9.999931
066-017,4.999997,4.999997
...,...,...
035-008,10.000032,10.000032
116-011,10.000000,10.000000
010-008,5.000043,5.000043
030-002,9.999999,9.999999


In [43]:
class AllSystem(object):
  def __init__(self,config):
    
    # C620 part
    self.c620_G = joblib.load(config['c620_G'])
    self.c620_F = joblib.load(config['c620_F'])
    self.c620_op_min = joblib.load(config['c620_op_min'])
    self.c620_op_max = joblib.load(config['c620_op_max'])
    
    # C660 part
    self.c660_G = joblib.load(config['c660_G'])
    self.c660_F = joblib.load(config['c660_F'])
    self.c660_op_min = joblib.load(config['c660_op_min'])
    self.c660_op_max = joblib.load(config['c660_op_max'])

    # C670 part
    self.c670_M = joblib.load(config['c670_M'])

    # columns name
    self.icg_col = joblib.load(config['icg_col_path'])
    self.c620_col = joblib.load(config['c620_col_path'])
    self.c660_col = joblib.load(config['c660_col_path'])
    self.c670_col = joblib.load(config['c670_col_path'])
    
    # other infomation
    self.c620_wt_always_same_split_factor_dict = joblib.load(config['c620_wt_always_same_split_factor_dict'])
    self.c660_wt_always_same_split_factor_dict = joblib.load(config['c660_wt_always_same_split_factor_dict'])
    self.c670_wt_always_same_split_factor_dict = joblib.load(config['c670_wt_always_same_split_factor_dict'])
    self.index_9999 = joblib.load(config['index_9999_path'])
    self.index_0001 = joblib.load(config['index_0001_path'])
    self.V615_density = 0.8626
    self.C820_density = 0.8731
    self.T651_density = 0.8749
  
  def inference(self,icg_input,c620_feed,t651_feed):
    idx = icg_input.index
    #c620
    c620_case = pd.DataFrame(index=idx,columns=self.c620_col['case'])
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'] = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values
    c620_op = self.c620_G.predict(c620_case.join(c620_feed))
    c620_sp = self.c620_F.predict(c620_case.join(c620_feed).join(c620_op))
    s1,s2,s3,s4 = c620_sp.iloc[:,:41].values,c620_sp.iloc[:,41:41*2].values,c620_sp.iloc[:,41*2:41*3].values,c620_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c620_feed,s1),sp2wt(c620_feed,s2),sp2wt(c620_feed,s3),sp2wt(c620_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c620_wt = pd.DataFrame(wt,index=idx,columns=self.c620_col['vent_gas_x']+self.c620_col['distillate_x']+self.c620_col['sidedraw_x']+self.c620_col['bottoms_x'])
    
    #c660
    V615_Btm_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr'].values.reshape(-1,1)
    C820_Dist_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr'].values.reshape(-1,1)
    V615_Btm_ton = V615_Btm_m3*self.V615_density
    C820_Dist_ton = C820_Dist_m3*self.C820_density
    c620_feed_rate_ton = V615_Btm_ton+C820_Dist_ton
    c620_mf_side = np.sum(c620_feed_rate_ton*c620_feed.values*s3*0.01,axis=1,keepdims=True)
    c620_mf_bot = np.sum(c620_feed_rate_ton*c620_feed.values*s4*0.01,axis=1,keepdims=True)
    t651_mf = (icg_input['Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr']*self.T651_density).values.reshape(-1,1)
    c660_mf = t651_mf + c620_mf_side
    t651_mf_p ,c620_mf_side_p = t651_mf/c660_mf ,c620_mf_side/c660_mf
    c660_feed = c620_wt[self.c620_col['sidedraw_x']].values*c620_mf_side_p + t651_feed.values*t651_mf_p
    c660_feed = pd.DataFrame(c660_feed,index=idx,columns=self.c660_col['x41'])
    c660_case = pd.DataFrame(index=idx,columns=self.c660_col['case'])
    c660_case['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'] = icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values
    c660_case['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'].values
    c660_op = self.c660_G.predict(c660_case.join(c660_feed))
    c660_sp = self.c660_F.predict(c660_case.join(c660_feed).join(c660_op))
    s1,s2,s3,s4 = c660_sp.iloc[:,:41].values,c660_sp.iloc[:,41:41*2].values,c660_sp.iloc[:,41*2:41*3].values,c660_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c660_feed,s1),sp2wt(c660_feed,s2),sp2wt(c660_feed,s3),sp2wt(c660_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c660_wt = pd.DataFrame(wt,index=idx,columns=self.c660_col['vent_gas_x']+self.c660_col['distillate_x']+self.c660_col['sidedraw_x']+self.c660_col['bottoms_x'])
    
    #c670
    c660_mf_bot = np.sum(c660_mf*c660_feed.values*s4*0.01,axis=1,keepdims=True)
    c670_mf = c620_mf_bot + c660_mf_bot
    c620_mf_bot_p,c660_mf_bot_p = c620_mf_bot/c670_mf , c660_mf_bot/c670_mf
    c670_feed = c620_wt[self.c620_col['bottoms_x']].values*c620_mf_bot_p + c660_wt[self.c660_col['bottoms_x']].values*c660_mf_bot_p
    c670_feed = pd.DataFrame(c670_feed,index=idx,columns=self.c670_col['combined'])
    c670_bf = pd.DataFrame(index=idx,columns=self.c670_col['upper_bf'])
    c620_bot_x = c620_wt[self.c620_col['bottoms_x']].values
    c660_bot_x = c660_wt[self.c660_col['bottoms_x']].values
    upper_bf = (c660_bot_x*c660_mf_bot)/(c620_bot_x*c620_mf_bot+c660_bot_x*c660_mf_bot)
    upper_bf = pd.DataFrame(upper_bf,index=idx,columns=self.c670_col['upper_bf'])
    upper_bf[list(set(self.index_9999)&set(upper_bf.columns))] = 0.9999
    upper_bf[list(set(self.index_0001)&set(upper_bf.columns))] = 0.0001
    c670_output = self.c670_M.predict(c670_feed.join(upper_bf))
    c670_sp,c670_op = c670_output.iloc[:,:41*2],c670_output.iloc[:,41*2:]    
    s1 = c670_sp[self.c670_col['distillate_sf']].values
    s2 = c670_sp[self.c670_col['bottoms_sf']].values
    w1 = sp2wt(c670_feed,s1)
    w2 = sp2wt(c670_feed,s2)
    c670_wt = np.hstack((w1,w2))
    c670_wt = pd.DataFrame(c670_wt,index = idx,columns=self.c670_col['distillate_x']+self.c670_col['bottoms_x'])
    return c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op
  
  def recommend(self,icg_input,c620_feed,t651_feed,search_iteration=300):
    idx = icg_input.index
    c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op = self.inference(icg_input,c620_feed,t651_feed)

    # c620 part
    c620_case = pd.DataFrame(index=idx,columns=self.c620_col['case'])
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'] = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values
    c620_op_col = c620_op.columns.tolist()
    def c620_objective(trial):
      c620_op_dict = {}
      for name in c620_op_col:
        c620_op_dict[name] = trial.suggest_uniform(name,self.c620_op_min[name],self.c620_op_max[name])
      c620_op = pd.DataFrame(c620_op_dict,index=idx)
      輸入端bz = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values[0]
      c620_sp = self.c620_F.predict(c620_case.join(c620_feed).join(c620_op))
      s1,s2,s3,s4 = c620_sp.iloc[:,:41].values,c620_sp.iloc[:,41:41*2].values,c620_sp.iloc[:,41*2:41*3].values,c620_sp.iloc[:,41*3:41*4].values
      w1,w2,w3,w4 = sp2wt(c620_feed,s1),sp2wt(c620_feed,s2),sp2wt(c620_feed,s3),sp2wt(c620_feed,s4)
      wt = np.hstack((w1,w2,w3,w4))
      c620_wt = pd.DataFrame(wt,index=idx,columns=self.c620_col['vent_gas_x']+self.c620_col['distillate_x']+self.c620_col['sidedraw_x']+self.c620_col['bottoms_x'])
      輸出端bz = c620_wt['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%'].values[0]
      loss = (輸入端bz - 輸出端bz)**2
      return loss
    study = optuna.create_study()
    study.optimize(c620_objective, n_trials=search_iteration)
    c620_op_opt = pd.DataFrame(study.best_params,index=idx)
    c620_op_delta = c620_op_opt - c620_op
    c620_sp = self.c620_F.predict(c620_case.join(c620_feed).join(c620_op_opt))
    s1,s2,s3,s4 = c620_sp.iloc[:,:41].values,c620_sp.iloc[:,41:41*2].values,c620_sp.iloc[:,41*2:41*3].values,c620_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c620_feed,s1),sp2wt(c620_feed,s2),sp2wt(c620_feed,s3),sp2wt(c620_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c620_wt = pd.DataFrame(wt,index=idx,columns=self.c620_col['vent_gas_x']+self.c620_col['distillate_x']+self.c620_col['sidedraw_x']+self.c620_col['bottoms_x'])
    輸入端bz = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values[0]
    輸出端bz = c620_wt['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%'].values[0]
    bz_error = abs(輸入端bz-輸出端bz)
    print('bz_error:',bz_error)
    
    # c660 part
    V615_Btm_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr'].values.reshape(-1,1)
    C820_Dist_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr'].values.reshape(-1,1)
    V615_Btm_ton = V615_Btm_m3*self.V615_density
    C820_Dist_ton = C820_Dist_m3*self.C820_density
    c620_feed_rate_ton = V615_Btm_ton+C820_Dist_ton
    c620_mf_side = np.sum(c620_feed_rate_ton*c620_feed.values*s3*0.01,axis=1,keepdims=True)
    c620_mf_bot = np.sum(c620_feed_rate_ton*c620_feed.values*s4*0.01,axis=1,keepdims=True)
    t651_mf = (icg_input['Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr']*self.T651_density).values.reshape(-1,1)
    c660_mf = t651_mf + c620_mf_side
    t651_mf_p ,c620_mf_side_p = t651_mf/c660_mf ,c620_mf_side/c660_mf
    c660_feed = c620_wt[self.c620_col['sidedraw_x']].values*c620_mf_side_p + t651_feed.values*t651_mf_p
    c660_feed = pd.DataFrame(c660_feed,index=idx,columns=self.c660_col['x41'])
    c660_case = pd.DataFrame(index=idx,columns=self.c660_col['case'])
    c660_case['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'] = icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']
    c660_case['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']
    c660_op_col = c660_op.columns.tolist()
    def c660_objective(trial):
      c660_op_dict = {}
      for name in c660_op_col:
        c660_op_dict[name] = trial.suggest_uniform(name,self.c660_op_min[name],self.c660_op_max[name])
      c660_op = pd.DataFrame(c660_op_dict,index=idx)
      輸入端nainbz = icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values[0]
      輸入端tol = icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'].values[0]
      c660_sp = self.c660_F.predict(c660_case.join(c660_feed).join(c660_op))
      s1,s2,s3,s4 = c660_sp.iloc[:,:41].values,c660_sp.iloc[:,41:41*2].values,c660_sp.iloc[:,41*2:41*3].values,c660_sp.iloc[:,41*3:41*4].values
      w1,w2,w3,w4 = sp2wt(c660_feed,s1),sp2wt(c660_feed,s2),sp2wt(c660_feed,s3),sp2wt(c660_feed,s4)
      wt = np.hstack((w1,w2,w3,w4))
      c660_wt = pd.DataFrame(wt,index=idx,columns=self.c660_col['vent_gas_x']+self.c660_col['distillate_x']+self.c660_col['sidedraw_x']+self.c660_col['bottoms_x'])
      na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
      輸出端nainbz = c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1).values[0]*10000
      輸出端tol = c660_wt['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%'].values[0]*10000
      loss1 = (輸入端nainbz - 輸出端nainbz)**2
      loss2 = (輸入端tol - 輸出端tol)**2
      return loss1+loss2
    study = optuna.create_study()
    study.optimize(c660_objective, n_trials=search_iteration)
    c660_op_opt = pd.DataFrame(study.best_params,index=idx)
    c660_op_delta = c660_op_opt - c660_op
    c660_sp = self.c660_F.predict(c660_case.join(c660_feed).join(c660_op_opt))
    s1,s2,s3,s4 = c660_sp.iloc[:,:41].values,c660_sp.iloc[:,41:41*2].values,c660_sp.iloc[:,41*2:41*3].values,c660_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c660_feed,s1),sp2wt(c660_feed,s2),sp2wt(c660_feed,s3),sp2wt(c660_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c660_wt = pd.DataFrame(wt,index=idx,columns=self.c660_col['vent_gas_x']+self.c660_col['distillate_x']+self.c660_col['sidedraw_x']+self.c660_col['bottoms_x'])
    輸入端nainbz = icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values[0]
    輸入端tol = icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'].values[0]
    na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
    輸出端nainbz = c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1).values[0]*10000
    輸出端tol = c660_wt['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%'].values[0]*10000
    nainbz_error = abs(輸入端nainbz-輸出端nainbz)
    tol_error = abs(輸入端tol-輸出端tol)
    print('nainbz_error:',nainbz_error)
    print('tol_error:',tol_error)

    # c670 part 
    
    c660_mf_bot = np.sum(c660_mf*c660_feed.values*s4*0.01,axis=1,keepdims=True)
    c670_mf = c620_mf_bot + c660_mf_bot
    c620_mf_bot_p,c660_mf_bot_p = c620_mf_bot/c670_mf , c660_mf_bot/c670_mf
    c670_feed = c620_wt[self.c620_col['bottoms_x']].values*c620_mf_bot_p + c660_wt[self.c660_col['bottoms_x']].values*c660_mf_bot_p
    c670_feed = pd.DataFrame(c670_feed,index=idx,columns=self.c670_col['combined'])
    c670_bf = pd.DataFrame(index=idx,columns=self.c670_col['upper_bf'])
    c620_bot_x = c620_wt[self.c620_col['bottoms_x']].values
    c660_bot_x = c660_wt[self.c660_col['bottoms_x']].values
    upper_bf = (c660_bot_x*c660_mf_bot)/(c620_bot_x*c620_mf_bot+c660_bot_x*c660_mf_bot)
    upper_bf = pd.DataFrame(upper_bf,index=idx,columns=self.c670_col['upper_bf'])
    upper_bf[list(set(self.index_9999)&set(upper_bf.columns))] = 0.9999
    upper_bf[list(set(self.index_0001)&set(upper_bf.columns))] = 0.0001
    c670_output = self.c670_M.predict(c670_feed.join(upper_bf))
    c670_sp,c670_op_opt = c670_output.iloc[:,:41*2],c670_output.iloc[:,41*2:]    
    s1 = c670_sp[self.c670_col['distillate_sf']].values
    s2 = c670_sp[self.c670_col['bottoms_sf']].values
    w1 = sp2wt(c670_feed,s1)
    w2 = sp2wt(c670_feed,s2)
    c670_wt = np.hstack((w1,w2))
    c670_wt = pd.DataFrame(c670_wt,index = idx,columns=self.c670_col['distillate_x']+self.c670_col['bottoms_x'])
    c670_op_delta = c670_op_opt - c670_op
    
    return c620_wt,c620_op_opt,c660_wt,c660_op_opt,c670_wt,c670_op_opt,bz_error,nainbz_error,tol_error

# 試算模式測試 觀察重點 準

In [44]:
f = AllSystem(config)
idx = np.random.choice(allidx,size=100,replace=False,p=None)
# minibatch input 
icg_input = icg_input.loc[idx]
c620_feed = c620_feed.loc[idx]
t651_feed = t651_feed.loc[idx]
# minibatch output 
c620_op = c620_op.loc[idx]
c620_wt = c620_wt.loc[idx]
c660_op = c660_op.loc[idx]
c660_wt = c660_wt.loc[idx]
c670_op = c670_op.loc[idx]
c670_wt = c670_wt.loc[idx]
c620_wt_,c620_op_,c660_wt_,c660_op_,c670_wt_,c670_op_ = f.inference(icg_input,c620_feed,t651_feed)
f.c670_M.show_metrics(c620_wt,c620_wt_,e=2e-2)

,R2,MSE,MAPE
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Methane_wt%,0.815615,0.00178881,2.11534
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,0.806518,0.495937,1.57252
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Propane_wt%,0.366997,0.0463187,0.351233
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,0.772144,0.220859,4.4904
...,...,...,...
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.999519,1.32304e-06,0.246213
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.99951,2.4402e-07,0.246213
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [45]:
f.c670_M.show_metrics(c620_op,c620_op_,e=2e-2)

,R2,MSE,MAPE
Density_Feed Properties,0.999761,3.46375e-10,0.0018433
Density_Vent Gas Production Rate and Composition,0.912564,1.13336e-06,0.203165
Density_Distillate Production Rate and Composition,0.708203,2.21151e-05,0.539967
Density_Sidedraw Production Rate and Composition,0.994662,5.76848e-09,0.00713196
Density_Bottoms Production Rate and Composition,0.996787,5.18647e-11,0.000697692
Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr,0.973481,4.1034,1.02374
Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.987082,0.0180593,0.726783
Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.982915,0.0339264,0.79244
Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,0.9861,0.0171108,0.0620211
Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC,0.984529,0.0115474,0.0452126


In [46]:
f.c670_M.show_metrics(c660_wt,c660_wt_,e=2e-2)

,R2,MSE,MAPE
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Methane_wt%,0.829774,0.0301407,11.5641
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,0.929486,2.69155,6.34907
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Propane_wt%,0.430408,3.2135,6.20539
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,0.889574,1.30365,14.278
...,...,...,...
Benzene Column C660 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.975749,2.53051e-25,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.976307,3.00266e-31,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [47]:
f.c670_M.show_metrics(c660_op,c660_op_,e=2e-2)

,R2,MSE,MAPE
Density_Feed Properties,0.986933,1.15345e-08,0.00896089
Density_Vent Gas Production Rate and Composition,0.952916,1.51751e-05,0.589465
Density_Distillate (Benzene Drag) Production Rate and Composition,0.960416,3.18889e-06,0.169249
Density_Sidedraw (Benzene )Production Rate and Composition,0.957182,3.75714e-12,0.000158015
Density_Bottoms Production Rate and Composition,0.993706,4.59434e-10,0.00195304
Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr,0.968216,25.4023,3.15578
Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.965775,0.209954,2.8954
Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.968643,0.2059,3.01456
Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,0.969055,0.00128586,0.0333634
Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC,0.962263,0.0292808,0.147058


In [48]:
f.c670_M.show_metrics(c670_wt,c670_wt_,e=2e-2)

,R2,MSE,MAPE
Toluene Column C670 Operation_Distillate Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Methane_wt%,0,1.68213e-09,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Ethane_wt%,0,1.06361e-06,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Propane_wt%,0,1.04833e-06,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_n-Butane_wt%,0,5.16427e-47,NaN
...,...,...,...
Toluene Column C670 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.999999,3.2785e-08,0.0207271
Toluene Column C670 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.999999,5.95077e-09,0.0207084
Toluene Column C670 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Toluene Column C670 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [49]:
f.c670_M.show_metrics(c670_op,c670_op_,e=2e-2)

,R2,MSE,MAPE
Density_Distillate Production Rate and Composition,0.995111,3.65557e-12,0.000172252
Density_Bottoms Production Rate and Composition,0.997051,5.00814e-10,0.00204126
Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,0.976607,12.8656,1.07245
Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.985133,0.129488,0.951245
Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.98251,0.141847,0.991215
Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,0.954599,0.00210527,0.0204789
Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC,0.986391,0.0384318,0.0694142
AVG,0.982486,1.88249,0.443859


# 推薦模式測試 直接給調幅 觀察重點調幅 已經是否滿足約束條件或足夠趨近約束條件

In [50]:
f = AllSystem(config)
icg_input = icg_input.iloc[[0]]
c620_feed = c620_feed.iloc[[0]]
t651_feed = t651_feed.iloc[[0]]
# minibatch output 
c620_op = c620_op.iloc[[0]]
c620_wt = c620_wt.iloc[[0]]
c660_op = c660_op.iloc[[0]]
c660_wt = c660_wt.iloc[[0]]
c670_op = c670_op.iloc[[0]]
c670_wt = c670_wt.iloc[[0]]

print(icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'])
print(icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'])
print(icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'])

# setting recommend value
icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'] = 70
icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'] = 980
icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = 10

print(icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'])
print(icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'])
print(icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'])


138-023    70.0
Name: Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%, dtype: float64
138-023    950.0
Name: Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw, dtype: float64
138-023    9.999948
Name: Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw, dtype: float64
138-023    70
Name: Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%, dtype: int64
138-023    980
Name: Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw, dtype: int64
138-023    10
Name: Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw, dtype: int64


In [51]:
c620_wt,c620_op_opt,c660_wt,c660_op_opt,c670_wt,c670_op_opt,bz_error,nainbz_error,tol_error = f.recommend(icg_input,c620_feed,t651_feed)

[I 2021-04-21 09:22:23,269] A new study created in memory with name: no-name-a5e2d495-d049-4c76-bc44-9ad4aecb1857
[I 2021-04-21 09:22:23,341] Trial 0 finished with value: 30.237387369279368 and parameters: {'Density_Feed Properties': 0.8634095466791304, 'Density_Vent Gas Production Rate and Composition': 0.4311352271511615, 'Density_Distillate Production Rate and Composition': 0.8119172114599982, 'Density_Sidedraw Production Rate and Composition': 0.8798497639919672, 'Density_Bottoms Production Rate and Composition': 0.8719622166298524, 'Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr': 145.81739637253708, 'Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr': 12.791051778629148, 'Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr': 15.11576146143395, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 175.76134899506337, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 19

bz_error: 0.009165700187480752


[I 2021-04-21 09:22:48,724] Trial 2 finished with value: 38515.81028109226 and parameters: {'Density_Feed Properties': 0.877077075707754, 'Density_Vent Gas Production Rate and Composition': 0.5259145952942406, 'Density_Distillate (Benzene Drag) Production Rate and Composition': 0.8689503529200334, 'Density_Sidedraw (Benzene )Production Rate and Composition': 0.8837466364945359, 'Density_Bottoms Production Rate and Composition': 0.8705382335582181, 'Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr': 75.35278298306979, 'Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr': 14.087987760919958, 'Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr': 15.147155542360164, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC': 87.14793898025819, 'Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC': 91.64498150898618}. Best is trial 1 with value: 1293.4862450888722.
[I 2021-04-21 09:22:48,791] Tria

nainbz_error: 0.2658598280551132
tol_error: 0.06592966372032016


# 確認三項條件有滿足

In [52]:
bz_error,nainbz_error,tol_error

(0.009165700187480752, 0.2658598280551132, 0.06592966372032016)

In [53]:
a = icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]
b = c620_wt[['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']]
a.join(b)

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
138-023,70,70.009166


In [54]:
a = icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]
b = c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,[1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] ].sum(axis=1)*10000
b.name = 'c660_wt NA in NA in Benzene_ppmw'
a.join(b)

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,c660_wt NA in NA in Benzene_ppmw
138-023,980,980.26586


In [55]:
a = icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]
b = c660_wt[['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%']]*10000
a.join(b)

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%
138-023,10,10.06593


# 確認操作條件有足夠大的調幅

In [56]:
c620_op_opt - c620_op

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate Production Rate and Composition,Density_Sidedraw Production Rate and Composition,Density_Bottoms Production Rate and Composition,Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr,Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
138-023,-0.000851,-0.016932,0.012074,0.001021,-0.000276,-26.602541,-2.099844,-1.335108,-1.54955,-0.683299


In [57]:
c660_op_opt - c660_op

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate (Benzene Drag) Production Rate and Composition,Density_Sidedraw (Benzene )Production Rate and Composition,Density_Bottoms Production Rate and Composition,Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr,Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC
138-023,0.003532,0.020106,-0.021434,0.000042,-0.000181,7.691558,-1.349659,-1.769655,-0.517426,-2.494919


In [58]:
c670_op_opt - c670_op

,Density_Distillate Production Rate and Composition,Density_Bottoms Production Rate and Composition,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
138-023,-0.000002,0.000049,-13.191528,-1.214851,-0.992209,-0.334091,-0.599915


# 確定以下三點誤差足夠小

In [59]:
bz_error

0.009165700187480752

In [60]:
nainbz_error

0.2658598280551132

In [61]:
tol_error

0.06592966372032016

# 保存樣本

In [62]:
idx = icg_input.index
print(idx)

Index(['138-023'], dtype='object')


In [63]:
demo = {
      # input
      'icg_input':icg_input.loc[idx],
      'c620_feed':c620_feed.loc[idx],
      't651_feed':t651_feed.loc[idx],
      # output
      'c620_op':c620_op.loc[idx],
      'c620_wt':c620_wt.loc[idx],
      'c660_op':c660_op.loc[idx],
      'c660_wt':c660_wt.loc[idx],
      'c670_op':c670_op.loc[idx],
      'c670_wt':c670_wt.loc[idx],
      }

In [64]:
import joblib
joblib.dump(demo,"/content/drive/MyDrive/台塑輕油案子/data/c620/demo/demo.pkl")

['/content/drive/MyDrive/台塑輕油案子/data/c620/demo/demo.pkl']

# 保存模組

In [65]:
joblib.dump(f,"/content/drive/MyDrive/台塑輕油案子/data/c620/model/allsystem.pkl")

['/content/drive/MyDrive/台塑輕油案子/data/c620/model/allsystem.pkl']